# NCA5 Hydrogen Senky Diagram
Looking at data from Princeton Net-Zero America: https://netzeroamerica.princeton.edu/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.sankey import Sankey

plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'

GWh_per_PJ = 277.778
TJ_per_PJ = 1000

In [2]:
# Take Princeton's input csv file and prep table with all needed values for Senky diagram.
def prep_dt(year=2050, scenario='E+RE+'):
    
    ### Read and get scenario
    print(f"\n\n{year}: {scenario}")
    df = pd.read_csv(f'data/nzap-data-{year}.csv')

    df = df[ df['scenario'] == scenario]
    
    
    
    ### Make new df for skimmed values
    cols = df.columns
    dic = {col : [] for col in cols}
    df1 = pd.DataFrame(dic)
    
    
    
    ### Skim all H2 related rows
    for idx in df.index:
        keep = False
        for col in cols:
            
            val = df.loc[idx, col]
            if type(val) == float or type(val) == np.float64 or type(val) == np.int64:
                continue
            
            if 'hydrogen' in val or 'Hydrogen' in val or 'H2' in val:
                keep = True
                break
        if keep:
            df1 = df1.append(df.loc[idx])
    
    df1.to_csv('tmp1.csv')
    
    
    
    ### Reduce to H2 energy related rows
    df1 = df1[ df1['unit'].isin(['PJ', 'TJ', 'GWh']) ]
    df1.to_csv('tmp2.csv')
    #print(df1)
    
    
    
    ### All values to GWh
    new_vals = []
    for idx in df1.index:
        val = df1.loc[idx, 'value']
        unit = df1.loc[idx, 'unit']
        if unit == 'GWh':
            new_vals.append(val)
        elif unit == 'PJ':
            new_vals.append(val * GWh_per_PJ)
        elif unit == 'TJ':
            new_vals.append(val / TJ_per_PJ * GWh_per_PJ)
        else:
            print(f"Unit was not considered in list, please fix this. Unit == {unit}")
            exit()
    df1['annual flow (GWh)'] = new_vals
    df1['annual flow (EJ)'] = df1['annual flow (GWh)'] / GWh_per_PJ / 1000.
    df1.to_csv('tmp3a.csv')
    #print(df1)

    
    
    ### Stacked bar comparison against PNZA's figures on slide 194 w/ production and use in EJ
    #stacked_plot(year, scenario, df1, 'Production')
    #stacked_plot(year, scenario, df1, 'Uses')
    
    

    ### Get conversion efficiencies and input energy
    effs = get_conversion_df()
    #print(effs)
    
    
    
    ### Map production conversion table to main data frame
    # NOTE: While SMR is in the datatable, SMR is skipped in the H2 production slides for 2050 (194)
    D = {
        'Production - BECCS-H2' : 'BECCS hydrogen production -> hydrogen blend',
        'Production - ATR' : 'autothermal reforming hydrogen production w/ccu -> hydrogen blend',
        'Production - Electrolysis' : 'central-station hydrogen electrolysis'
    }
    # Silly difference in name nomenclature
    S = {
        'E+RE+' : 'E+ RE+',
        'E+RE-' : 'E+ RE-',
        'REF' : 'REF',
    }
    assert(scenario in S.keys())
    
    
    ### Create main table
    # Input energy types
    inputs = []
    for tech, val in D.items():
        #print(tech)
        for idx in effs.index:
            T = effs.loc[idx, 'tech']
            #print(T)
            if T != val:
                continue
            if effs.loc[idx, 'unit'] != 'energy in/energy out':
                continue
            tech_in = effs.loc[idx, 'blend_in']
            if tech_in not in inputs:
                inputs.append(tech_in)
    print(f"Input techs: {inputs}")
    
    # Dict for new DF
    input_d = {
        'tech' : ['' for _ in range(len(D.keys()))],
        'H2 production (GWh)' : [0 for _ in range(len(D.keys()))],
    }
    for IN in inputs:
        input_d[f"{IN} (in/out)"] = [0 for _ in range(len(D.keys()))]
    
    # Populate dic
    i = 0
    for tech, val in D.items():
        input_d['tech'][i] = tech
        for idx in effs.index:
            T = effs.loc[idx, 'tech']
            #print(T)
            if T != val:
                continue
            if effs.loc[idx, 'unit'] != 'energy in/energy out':
                continue
            tech_in = effs.loc[idx, 'blend_in']
            conversion = effs.loc[idx, S[scenario]]
            input_d[f"{tech_in} (in/out)"][i] = conversion
        
        # Get row with current tech (ensure only 1 row is selected)
        row = df1['variable_name'] == tech
        assert(row.sum() == 1)
        prod = df1.loc[ row, 'annual flow (GWh)'].sum()
        input_d['H2 production (GWh)'][i] = prod
        
        i += 1
    dfx = pd.DataFrame(input_d)
    
    # Calculate total input energy
    dfx['input (GWh)'] = np.zeros(len(dfx.index))
    dfx['electricity output (GWh)'] = np.zeros(len(dfx.index))
    for IN in inputs:
        dfx[f"{IN} (GWh input)"] = np.where(dfx[f"{IN} (in/out)"] >= 0, dfx[f"{IN} (in/out)"], 0)  * dfx['H2 production (GWh)']
        dfx['electricity output (GWh)'] += np.where(dfx[f"{IN} (in/out)"] < 0, -1 * dfx[f"{IN} (in/out)"], 0)  * dfx['H2 production (GWh)']
        dfx['input (GWh)'] += dfx[f"{IN} (GWh input)"]
    dfx['losses (GWh)'] = dfx['input (GWh)'] - dfx['H2 production (GWh)'] - dfx['electricity output (GWh)']
    dfx['eff (%)'] = (dfx['H2 production (GWh)'] + dfx['electricity output (GWh)']) / dfx['input (GWh)'] * 100
    
    # Print summary outputs
    
    
    
    
    return dfx


def stacked_plot(year, scenario, df, col):
    fig, ax = plt.subplots()
    plt.title(f"{year}, {scenario}: {col}")
    
    df1a = df[ df['filter_level_3'] == col ]
    df1a = df1a.set_index('variable_name')
    dT = df1a[['annual flow (EJ)',]].T
    dT.plot(kind='bar', stacked=True, ax=ax)

    
    #d.plot(kind='bar', ax=f.gca())
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    ax.set_ylabel('H2 Quantity (EJ)')
    ax.xaxis.set_ticklabels([])
    plt.tight_layout()
    plt.savefig(f"plots/{year}_{scenario}_{col}.png")
    plt.show()
    return


def get_conversion_df():
    effs = pd.read_excel('data/NZA_Annex_A3_-_Inputs_catalog_for_EER_modeling.xlsx', sheet_name='conversion_efficiency')
    
    # fill in NaNs from previously merged cells
    p_dic = {}
    for col in ['tech', 'zone', 'vintage', 'unit']:
        p_dic[col] = ''
    for idx in effs.index:
        for col in p_dic.keys():
            val = effs.loc[idx, col]
            if type(val) != str and np.isnan(val):
                effs.loc[idx, col] = p_dic[col]
                #print('replaced', val, type(val))
            else:
                p_dic[col] = val
    
    return effs
    
    
    

#dfx = prep_dt(2050, 'E+RE+')
#dfx = prep_dt(2050, 'E+RE-')

In [3]:
def sources_to_prod(df, year, scenario):
    
    input_cols = [col for col in df.columns if '(GWh input)' in col]
    input_cols.insert(0, 'tech')
    #print(input_cols)
    df1 = df[input_cols]
    #print(df1)
    return df1

def prod_to_output(df, year, scenario):
    
    output_cols = ['tech', 'electricity output (GWh)', 'H2 production (GWh)', 'losses (GWh)']
    #print(output_cols)
    df1 = df[output_cols]
    #print(df1)
    return df1

def output_to_uses(year, scenario):
    
    df = pd.read_csv(f'data/nzap-data-{year}.csv')
    df = df[ df['scenario'] == scenario]
    
    df1 = cluster_uses(df)
    df1['electricity use (GWh)'] = np.zeros(len(df1.index))
    df1['losses (GWh)'] = np.zeros(len(df1.index))
    return df1

def cluster_uses(df):
    
    df1 = df[ (df['filter_level_2'] == 'Hydrogen') & (df['filter_level_3'] == 'Uses')]
    #print(df1['variable_name'])
    #print(df1)
    clustering = {
        'Uses - Bulk chemicals manufacturing' : 'chemicals manufacturing',
        'Uses - Direct-reduced iron production' : 'steel',
        'Uses - Gas turbine fuel' : 'electricity',
        'Uses - Gaseous fuels synthesis' : 'fuel synthesis',
        'Uses - Hythane' : 'other',
        'Uses - Industrial boilers' : 'industrial heat',
        'Uses - Liquid fuels synthesis' : 'fuel synthesis',
        'Uses - Medium and heavy fuel cell vehicles' : 'transportation',
        'Uses - Other industry' : 'other',
        'Uses - Other transportation' : 'transportation',
    }
    uses = []
    for k, v in clustering.items():
        if v not in uses:
            uses.append(v)
    vals = np.zeros(len(uses))
    df2 = pd.DataFrame({'h2 use (GWh)' : vals}, index=uses)
    #tot = 0
    for idx in df1.index:
        use = df1.loc[idx, 'variable_name']
        #print(use, clustering[use])
        #print(df2.loc[clustering[use], 'h2 use (GWh)'])
        df2.loc[clustering[use], 'h2 use (GWh)'] += df1.loc[idx, 'value']
        #tot += df1.loc[idx, 'value']
    print(df2)
    #print(tot, df2['h2 use (GWh)'].sum())
    return df2
    
year = 2050
scenario = 'E+RE+'
dfx = prep_dt(year, scenario)
s_to_p = sources_to_prod(dfx, year, scenario)
p_to_o = prod_to_output(dfx, year, scenario)
o_to_u = output_to_uses(year, scenario)

with pd.ExcelWriter(f'output_{year}_{scenario}.xlsx') as writer:  
    s_to_p.to_excel(writer, sheet_name='Source_to_Production', index=False)
    p_to_o.to_excel(writer, sheet_name='Production_to_Output', index=False)
    o_to_u.to_excel(writer, sheet_name='Output_to_Uses')



2050: E+RE+
Input techs: ['biomass blend - solids', 'electricity', 'pipeline gas blend']
                         h2 use (GWh)
chemicals manufacturing  2.961703e+05
steel                    1.202949e+05
electricity              6.889429e+04
fuel synthesis           4.220614e+06
other                    1.369115e+05
industrial heat          1.069906e+04
transportation           5.000128e+05
